## Frontier Model for generating code.


In [32]:
# imports
import os
import io
import sys
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
from anthropic import Anthropic
from IPython.display import Markdown, display, update_display
import gradio as gr
import subprocess


load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")

os.environ["PATH"] += r";C:\msys64\mingw64\bin"

if openai_api_key.startswith("sk-proj") and len(openai_api_key) > 10:
    print("OpenAI api key is correct and ready for use!")
else:
    print("OpenAI api key is incorrect! Please provide a functional one for the best possible quality.")

if anthropic_api_key.startswith("sk-ant") and len(anthropic_api_key) > 10:
    print("Anthropic api key is correct and ready for use!")
else:
    print("Anthropic api key is incorrect! Please provide a functional one for the best possible quality.")


openai = OpenAI()
claude = Anthropic()


OPENAI_MODEL = "gpt-4o-mini"
CLAUDE_MODEL = "claude-3-haiku-20240307"


OpenAI api key is correct and ready for use!
Anthropic api key is correct and ready for use!


In [13]:
system_message = "You are an assistant that reimplements Python code in high performance C++ for a HP Elitebook I5 running on Windows 10. "
system_message += "Respond only with C++ code; use comments sparingly and do not provide any explanation other than occasional comments. "
system_message += "The C++ response needs to produce an identical output in the fastest possible time."

In [33]:
def user_prompt_for(python):
    user_prompt = "Rewrite this Python code in C++ with the fastest possible implementation that produces identical output in the least time. "
    user_prompt += "Respond only with C++ code; do not explain your work other than a few comments. "
    user_prompt += "Pay attention to number types to ensure no int overflows. Remember to #include all necessary C++ packages such as iomanip.\n\n"
    user_prompt += python
    return user_prompt

def messages_for(python):
    return [
        {"role" : "system", "content" : system_message},
        {"role" : "user", "content" : user_prompt_for(python)}
    ]


def write_output(cpp):
    code = cpp.replace("```cpp","").replace("```","")
    with open("optimized.cpp",'w') as file:
        file.write(code)


def optimize_gpt(python):
    stream = openai.chat.completions.create(
        model = OPENAI_MODEL,
        messages = messages_for(python),
        stream = True
    )

    response = ""
    for chunk in stream:
        part = chunk.choices[0].delta.content or ""
        response += part
        print(part, end = '', flush = True)

    write_output(response)


def optimize_claude(python):
    response = claude.messages.stream(
        model = CLAUDE_MODEL,
        max_tokens = 2000,
        system = system_message,
        messages = [{"role" : "user", "content" : user_prompt_for(python)}]
    )

    reply = ""
    with response as stream:
        for part in stream.text_stream:
            reply += part
            print(part, end = '', flush = True)

    write_output(reply)


In [23]:
pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(100_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [24]:
exec(pi)

Result: 3.141592658589
Execution Time: 20.806433 seconds


In [25]:
optimize_gpt(pi)

```cpp
#include <iostream>
#include <iomanip>
#include <chrono>

double calculate(long long iterations, double param1, double param2) {
    double result = 1.0;
    for (long long i = 1; i <= iterations; ++i) {
        double j1 = i * param1 - param2;
        result -= (1.0 / j1);
        double j2 = i * param1 + param2;
        result += (1.0 / j2);
    }
    return result;
}

int main() {
    auto start_time = std::chrono::high_resolution_clock::now();
    double result = calculate(100000000, 4.0, 1.0) * 4.0;
    auto end_time = std::chrono::high_resolution_clock::now();

    std::cout << std::fixed << std::setprecision(12) << "Result: " << result << std::endl;
    std::chrono::duration<double> elapsed = end_time - start_time;
    std::cout << "Execution Time: " << std::fixed << std::setprecision(6) << elapsed.count() << " seconds" << std::endl;

    return 0;
}
```

In [26]:
!clang++ -O3 -std=c++17 -o optimized.exe optimized.cpp
!optimized.exe

Result: 3.141592658589
Execution Time: 0.195461 seconds


In [34]:
optimize_claude(pi)

```cpp
#include <iostream>
#include <iomanip>
#include <chrono>

double calculate(long long iterations, double param1, double param2) {
    double result = 1.0;
    for (long long i = 1; i <= iterations; i++) {
        double j = i * param1 - param2;
        result -= (1.0 / j);
        j = i * param1 + param2;
        result += (1.0 / j);
    }
    return result;
}

int main() {
    long long iterations = 100000000LL;
    double param1 = 4.0, param2 = 1.0;

    auto start_time = std::chrono::high_resolution_clock::now();
    double result = calculate(iterations, param1, param2) * 4.0;
    auto end_time = std::chrono::high_resolution_clock::now();

    std::cout << std::fixed << std::setprecision(12) << "Result: " << result << std::endl;
    std::cout << std::fixed << std::setprecision(6) << "Execution Time: " << std::chrono::duration<double>(end_time - start_time).count() << " seconds" << std::endl;

    return 0;
}
```

In [35]:
!clang++ -O3 -std=c++17 -o optimized.exe optimized.cpp
!optimized.exe

Result: 3.141592658589
Execution Time: 0.197099 seconds
